- DGL NN 模块是用户构建 GNN 模型的基本模块。
- NN 模块的父类取决于后端所使用的深度神经网络框架。(对于 PyTorch 后端， 它应该继承 PyTorch 的 NN 模块)

## DGL NN模块的构造函数

构造函数完成以下几个任务：

1. 设置选项。

2. 注册可学习的参数或者子模块。

3. 初始化参数。

In [1]:
import torch.nn as nn

from dgl.utils import expand_as_pair

class SAGEConv(nn.Module):
    def __init__(self,
                 in_feats,
                 out_feats,
                 aggregator_type,
                 bias=True,
                 norm=None,
                 activation=None):
        super(SAGEConv, self).__init__()

        self._in_src_feats, self._in_dst_feats = expand_as_pair(in_feats)
        self._out_feats = out_feats
        self._aggre_type = aggregator_type
        self.norm = norm
        self.activation = activation

- 对于图神经网络，输入维度可被分为源节点特征维度和目标节点特征维度。
- 聚合类型 (self._aggre_type)。对于特定目标节点，聚合类型决定了如何聚合不同边上的信息。(mean ,sum ,max,min ,lstm)
- norm 是用于特征归一化的可调用函数。在 SAGEConv 论文里，归一化可以是 L2 归一化: $hv=hv/∥hv∥_2$。

In [ ]:
# 聚合类型：mean、pool、lstm、gcn
if aggregator_type not in ['mean', 'pool', 'lstm', 'gcn']:
    raise KeyError('Aggregator type {} not supported.'.format(aggregator_type))
if aggregator_type == 'pool':
    self.fc_pool = nn.Linear(self._in_src_feats, self._in_src_feats)
if aggregator_type == 'lstm':
    self.lstm = nn.LSTM(self._in_src_feats, self._in_src_feats, batch_first=True)
if aggregator_type in ['mean', 'pool', 'lstm']:
    self.fc_self = nn.Linear(self._in_dst_feats, out_feats, bias=bias)
self.fc_neigh = nn.Linear(self._in_src_feats, out_feats, bias=bias)
self.reset_parameters()

def reset_parameters(self):
    """重新初始化可学习的参数"""
    gain = nn.init.calculate_gain('relu')
    if self._aggre_type == 'pool':
        nn.init.xavier_uniform_(self.fc_pool.weight, gain=gain)
    if self._aggre_type == 'lstm':
        self.lstm.reset_parameters()
    if self._aggre_type != 'gcn':
        nn.init.xavier_uniform_(self.fc_self.weight, gain=gain)
    nn.init.xavier_uniform_(self.fc_neigh.weight, gain=gain)

## 3.2 编写 DGL NN 模块的 forward 函数

- forward() 函数执行了实际的消息传递和计算。
- 与通常以张量为参数的 PyTorch NN 模块相比,DGL NN 模块额外增加了 1 个参数 dgl.DGLGraph
- forward()函数一般操作
  1. 检测输入图对象是否符合规范。
  2. 消息传递和聚合。
  3. 聚合后，更新特征作为输出

### 输入图对象的规范检测


In [2]:
def forward(self, graph, feat):
    with graph.local_scope():
        # 指定图类型，然后根据图类型扩展输入特征
        feat_src, feat_dst = expand_as_pair(feat, graph)

SAGEConv数学公式

$$
\begin{gathered}
h_{\mathcal{N}(d s t)}^{(l+1)}=\operatorname{aggregate}\left(\left\{h_{s r c}^l, \forall s r c \in \mathcal{N}(d s t)\right\}\right) \\
h_{d s t}^{(l+1)}=\sigma\left(W \cdot \operatorname{concat}\left(h_{d s t}^l, h_{\mathcal{N}(d s t)}^{l+1}\right)+b\right) \\
h_{d s t}^{(l+1)}=\operatorname{norm}\left(h_{d s t}^{l+1}\right)
\end{gathered}
$$

- 源节点特征 feat_src 和目标节点特征 feat_dst 需要根据图类型被指定。 
-  feat 扩展为 feat_src 和 feat_dst 的函数是 expand_as_pair()。

In [3]:
def expand_as_pair(input_, g=None):
    if isinstance(input_, tuple):
        # 二分图的情况
        return input_
    elif g is not None and g.is_block:
        # 子图块的情况
        if isinstance(input_, Mapping):
            input_dst = {
                k: F.narrow_row(v, 0, g.number_of_dst_nodes(k))
                for k, v in input_.items()}
        else:
            input_dst = F.narrow_row(input_, 0, g.number_of_dst_nodes())
        return input_, input_dst
    else:
        # 同构图的情况
        return input_, input_

### 消息传递和聚合

In [ ]:
import dgl.function as fn
import torch.nn.functional as F
from dgl.utils import check_eq_shape

if self._aggre_type == 'mean':
    graph.srcdata['h'] = feat_src
    graph.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'neigh'))
    h_neigh = graph.dstdata['neigh']
elif self._aggre_type == 'gcn':
    check_eq_shape(feat)
    graph.srcdata['h'] = feat_src
    graph.dstdata['h'] = feat_dst
    graph.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'neigh'))
    # 除以入度
    degs = graph.in_degrees().to(feat_dst)
    h_neigh = (graph.dstdata['neigh'] + graph.dstdata['h']) / (degs.unsqueeze(-1) + 1)
elif self._aggre_type == 'pool':
    graph.srcdata['h'] = F.relu(self.fc_pool(feat_src))
    graph.update_all(fn.copy_u('h', 'm'), fn.max('m', 'neigh'))
    h_neigh = graph.dstdata['neigh']
else:
    raise KeyError('Aggregator type {} not recognized.'.format(self._aggre_type))

# GraphSAGE中gcn聚合不需要fc_self
if self._aggre_type == 'gcn':
    rst = self.fc_neigh(h_neigh)
else:
    rst = self.fc_self(h_self) + self.fc_neigh(h_neigh)

### 聚合后，更新特征作为输出

In [ ]:
# 激活函数
if self.activation is not None:
    rst = self.activation(rst)
# 归一化
if self.norm is not None:
    rst = self.norm(rst)
return rst

## 3.3 异构图上的 GraphConv 模块

HeteroGraphConv，用于定义异构图上 GNN 模块。 实现逻辑与消息传递级别的 API multi_update_all() 相同

- 每个关系上的 DGL NN 模块。

- 聚合来自**不同关系**上的结果。

$$
h_{d s t}^{(l+1)}=\underset{r \in \mathcal{R}, r_{d s F} t d s t}{A G G}\left(f_r\left(g_r, h_{r s r c}^l, h_{r d s t}^l\right)\right)
$$

其中 $f_r$ 是对应每个关系 $r$ 的 NN 模块，AGG 是聚合函数。

In [ ]:
import torch.nn as nn

class HeteroGraphConv(nn.Module):
    def __init__(self, mods, aggregate='sum'): #mod关系名，值为作用在该关系上 NN 模块对象。
        super(HeteroGraphConv, self).__init__()
        self.mods = nn.ModuleDict(mods)
        if isinstance(aggregate, str):
            # 获取聚合函数的内部函数
            self.agg_fn = get_aggregate_fn(aggregate)
        else:
            self.agg_fn = aggregate
    def forward(self, g, inputs, mod_args=None, mod_kwargs=None):#mod_args 和 mod_kwargs。 这 2 个字典与 self.mods 具有相同的键，值则为对应 NN 模块的自定义参数。
    if mod_args is None:
        mod_args = {}
    if mod_kwargs is None:
        mod_kwargs = {}
    outputs = {nty : [] for nty in g.dsttypes}